In [ ]:
import requests
import pandas as pd
import time

def fetch_kucoin_prices(symbol, timestamps):
    """Fetches historical spot and futures closing prices from KuCoin."""
    spot_prices = {}
    futures_prices = {}
    
    for ts in timestamps:
        unix_ts = int(time.mktime(ts.timetuple()))  # Convert to Unix timestamp
        
        # Fetch spot price
        spot_url = "https://api.kucoin.com/api/v1/market/candles"
        spot_params = {"symbol": symbol, "type": "1day", "startAt": unix_ts, "endAt": unix_ts + 86400}
        spot_resp = requests.get(spot_url, params=spot_params).json()
        
        if spot_resp.get("data"):
            spot_prices[ts] = spot_resp["data"][0][2]  # Close price (index 2)
        
        # Fetch futures price
        futures_url = "https://api-futures.kucoin.com/api/v1/kline/query"
        futures_params = {"symbol": symbol + "M", "granularity": 1440, "from": unix_ts, "to": unix_ts + 86400}
        futures_resp = requests.get(futures_url, params=futures_params).json()
        
        if futures_resp.get("data"):
            futures_prices[ts] = futures_resp["data"][0][2]  # Close price (index 2)
        
        time.sleep(0.5)  # Avoid rate limits
    
    return spot_prices, futures_prices

# Load the funding history CSV
file_path = "/Users/juninho/School/UOLY2/External Project/Btech-Trading-Bot/Arbitrage/dataset/kucoin_funding_history.csv"
funding_df = pd.read_csv(file_path)
funding_df.columns = funding_df.columns.str.strip()
funding_df = funding_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
funding_df['Time'] = pd.to_datetime(funding_df['Time'], format='%m/%d/%Y %H:%M:%S', errors='coerce')

# Fetch prices
symbol = "BTC-USDT"  # Modify if needed
timestamps = funding_df['Time'].tolist()
spot_prices, futures_prices = fetch_kucoin_prices(symbol, timestamps)

# Add prices to DataFrame
funding_df['Spot Closing Price'] = funding_df['Time'].map(spot_prices)
funding_df['Futures Closing Price'] = funding_df['Time'].map(futures_prices)

# Save to new CSV
output_path = "kucoin_funding_with_prices.csv"
funding_df.to_csv(output_path, index=False)
print(f"Data saved to {output_path}")


/var/folders/rl/039_ddsn1zjfd2t00_pz697w0000gn/T/ipykernel_61019/1619387592.py:37: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  funding_df = funding_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


Data saved to kucoin_funding_with_prices.csv


In [11]:
import requests
import pandas as pd
import time

def fetch_kucoin_prices(symbol, timestamps):
    """Fetches historical spot and futures closing prices from KuCoin."""
    spot_prices = {}
    futures_prices = {}
    
    for ts in timestamps:
        unix_ts = int(time.mktime(ts.timetuple()))  # Convert to Unix timestamp
        
        # Fetch spot price (8-hour interval)
        spot_url = "https://api.kucoin.com/api/v1/market/candles"
        spot_params = {"symbol": symbol, "type": "8hour", "startAt": unix_ts, "endAt": unix_ts + 28800}
        spot_resp = requests.get(spot_url, params=spot_params).json()
        
        if spot_resp.get("data"):
            spot_prices[ts] = spot_resp["data"][0][2]  # Close price (index 2)
        
        # Fetch futures price (8-hour interval)
        futures_url = "https://api-futures.kucoin.com/api/v1/kline/query"
        futures_params = {"symbol": symbol.replace("-", "") + "M", "granularity": 480, "from": unix_ts, "to": unix_ts + 28800}
        futures_resp = requests.get(futures_url, params=futures_params).json()
        
        if futures_resp.get("data"):
            futures_prices[ts] = futures_resp["data"][0][2]  # Close price (index 2)
        
        time.sleep(0.5)  # Avoid rate limits
    
    return spot_prices, futures_prices

# Load the funding history CSV
file_path = "/Users/juninho/School/UOLY2/External Project/Btech-Trading-Bot/Arbitrage/Merged_Funding_History.csv"
funding_df = pd.read_csv(file_path)
funding_df.columns = funding_df.columns.str.strip()
funding_df = funding_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
funding_df['Time'] = pd.to_datetime(funding_df['Time'], format='%m/%d/%Y %H:%M:%S', errors='coerce')

# Fetch prices
symbol = "BTC-USDT"  # Modify if needed
timestamps = funding_df['Time'].tolist()
spot_prices, futures_prices = fetch_kucoin_prices(symbol, timestamps)

# Add prices to DataFrame
funding_df['Spot Closing Price'] = funding_df['Time'].map(spot_prices)
funding_df['Futures Closing Price'] = funding_df['Time'].map(futures_prices)

# Save to new CSV
output_path = "kucoin_funding_with_prices.csv"
funding_df.to_csv(output_path, index=False)
print(f"Data saved to {output_path}")


/var/folders/rl/039_ddsn1zjfd2t00_pz697w0000gn/T/ipykernel_61019/3085629859.py:37: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  funding_df = funding_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


Data saved to kucoin_funding_with_prices.csv


In [15]:
import time
import ccxt
import pandas as pd

def fetch_kucoin_futures_prices(symbol, timestamps):
    """Fetches 8-hour futures closing prices from KuCoin."""
    futures_prices = {}
    exchange = ccxt.kucoinfutures()
    
    for ts in timestamps:
        try:
            since = int(ts.timestamp() * 1000)  # Convert to milliseconds
            
            # Fetch 8-hour kline data from KuCoin Futures
            ohlcv = exchange.fetch_ohlcv(symbol, '8h', since, 1)
            if ohlcv:
                futures_prices[ts] = ohlcv[0][4]  # Close price (index 4)
            
            time.sleep(0.5)  # Avoid rate limits
        except Exception as e:
            print(f"Error fetching futures price for {ts}: {e}")
    
    return futures_prices

# Load the funding history CSV
file_path = "/Users/juninho/School/UOLY2/External Project/Btech-Trading-Bot/Arbitrage/kucoin_funding_with_prices.csv"
funding_df = pd.read_csv(file_path)
funding_df.columns = funding_df.columns.str.strip()
funding_df = funding_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
funding_df['Time'] = pd.to_datetime(funding_df['Time'], errors='coerce')

# Remove NaT values from timestamps
timestamps = funding_df['Time'].dropna().tolist()

# Initialize CCXT KuCoin Futures exchange
exchange = ccxt.kucoinfutures()

def format_kucoin_symbol(spot_symbol):
    """Converts a spot trading pair into the correct KuCoin Futures format."""
    return spot_symbol.replace("-", "/") + ":USDTM"  # Ensure correct symbol format

# Define the correct futures symbol
symbol = "BTC-USDT"
futures_symbol = format_kucoin_symbol(symbol)
print(f"Fetching futures data for symbol: {futures_symbol}")

# Fetch futures closing prices
futures_prices = fetch_kucoin_futures_prices(futures_symbol, timestamps)

# Map futures prices to the DataFrame
funding_df['Futures Closing Price'] = funding_df['Time'].map(futures_prices)

# Save to new CSV
output_path = "kucoin_funding_with_prices.csv"
funding_df.to_csv(output_path, index=False)
print(f"Data saved to {output_path}")

/var/folders/rl/039_ddsn1zjfd2t00_pz697w0000gn/T/ipykernel_61019/863257946.py:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  funding_df = funding_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


Fetching futures data for symbol: BTC/USDT:USDTM
Error fetching futures price for 2025-03-01 20:00:00: kucoinfutures does not have market symbol BTC/USDT:USDTM
Error fetching futures price for 2025-03-01 12:00:00: kucoinfutures does not have market symbol BTC/USDT:USDTM
Error fetching futures price for 2025-03-01 04:00:00: kucoinfutures does not have market symbol BTC/USDT:USDTM
Error fetching futures price for 2025-02-28 20:00:00: kucoinfutures does not have market symbol BTC/USDT:USDTM
Error fetching futures price for 2025-02-28 12:00:00: kucoinfutures does not have market symbol BTC/USDT:USDTM
Error fetching futures price for 2025-02-28 04:00:00: kucoinfutures does not have market symbol BTC/USDT:USDTM
Error fetching futures price for 2025-02-27 20:00:00: kucoinfutures does not have market symbol BTC/USDT:USDTM
Error fetching futures price for 2025-02-27 12:00:00: kucoinfutures does not have market symbol BTC/USDT:USDTM
Error fetching futures price for 2025-02-27 04:00:00: kucoinfut

In [16]:
import time
import ccxt
import pandas as pd
from datetime import datetime

def fetch_kucoin_futures_prices(symbol, timestamps):
    """Fetches 8-hour futures closing prices from KuCoin."""
    futures_prices = {}
    exchange = ccxt.kucoinfutures({
        'enableRateLimit': True  # Enable built-in rate limiting
    })
    
    print(f"Available symbols: {exchange.load_markets().keys()}")
    
    for ts in timestamps:
        try:
            since = int(ts.timestamp() * 1000)  # Convert to milliseconds
            
            # Fetch 8-hour kline data from KuCoin Futures
            ohlcv = exchange.fetch_ohlcv(symbol, '8h', since, 1)
            if ohlcv and len(ohlcv) > 0:
                futures_prices[ts] = ohlcv[0][4]  # Close price (index 4)
                print(f"Found price for {ts}: {ohlcv[0][4]}")
            else:
                print(f"No data returned for {ts}")
            
            time.sleep(1)  # Longer sleep to avoid rate limits
        except Exception as e:
            print(f"Error fetching futures price for {ts}: {e}")
    
    return futures_prices

# Load the funding history CSV
file_path = "/Users/juninho/School/UOLY2/External Project/Btech-Trading-Bot/Arbitrage/kucoin_funding_with_prices.csv"
funding_df = pd.read_csv(file_path)
funding_df.columns = funding_df.columns.str.strip()
funding_df = funding_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
funding_df['Time'] = pd.to_datetime(funding_df['Time'], errors='coerce')

# Remove NaT values from timestamps
timestamps = funding_df['Time'].dropna().tolist()

# Initialize CCXT KuCoin Futures exchange
exchange = ccxt.kucoinfutures()

# Print available markets to help identify the correct symbol
markets = exchange.load_markets()
print("Available futures markets:")
btc_markets = [symbol for symbol in markets.keys() if 'BTC' in symbol]
print(btc_markets)

# Check which symbol format is correct for BTC-USDT futures
# Common formats for BTC futures on KuCoin are:
futures_symbol = 'XBTUSDTM'  # Perpetual contract for BTC
# Alternative options if the above doesn't work:
# futures_symbol = 'BTC-USDT-SWAP'
# futures_symbol = 'BTCUSDTM'

print(f"Fetching futures data for symbol: {futures_symbol}")

# Fetch futures closing prices
futures_prices = fetch_kucoin_futures_prices(futures_symbol, timestamps)

# Map futures prices to the DataFrame
funding_df['Futures Closing Price'] = funding_df['Time'].map(futures_prices)

# Save to new CSV
output_path = "kucoin_funding_with_future_prices.csv"
funding_df.to_csv(output_path, index=False)
print(f"Data saved to {output_path}")

/var/folders/rl/039_ddsn1zjfd2t00_pz697w0000gn/T/ipykernel_61019/3232868620.py:37: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  funding_df = funding_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


Available futures markets:
['BTC/USDT:USDT', 'BTC/USD:BTC-250328', 'BTC/USDC:USDC', 'BTC/USD:BTC']
Fetching futures data for symbol: XBTUSDTM
Available symbols: dict_keys(['BTC/USDT:USDT', 'ETH/USDT:USDT', 'SOL/USDT:USDT', 'WIF/USDT:USDT', 'PEPE/USDT:USDT', 'DOGE/USDT:USDT', 'XRP/USDT:USDT', '1000000MOG/USDT:USDT', '10000CAT/USDT:USDT', '10000COQ/USDT:USDT', '10000LADYS/USDT:USDT', '10000SATS/USDT:USDT', '10000WEN/USDT:USDT', '1000BONK/USDT:USDT', '1000CHEEMS/USDT:USDT', '1000RATS/USDT:USDT', '1000X/USDT:USDT', '1INCH/USDT:USDT', '1MBABYDOGE/USDT:USDT', 'AAVE/USDT:USDT', 'ACE/USDT:USDT', 'ACH/USDT:USDT', 'ACTSOL/USDT:USDT', 'ADA/USDT:USDT', 'AERO/USDT:USDT', 'AEVO/USDT:USDT', 'AGI/USDT:USDT', 'AGLD/USDT:USDT', 'AI16Z/USDT:USDT', 'AI/USDT:USDT', 'AIXBT/USDT:USDT', 'ALCH/USDT:USDT', 'ALGO/USDT:USDT', 'ALICE/USDT:USDT', 'ALPACA/USDT:USDT', 'ALPHA/USDT:USDT', 'APTOSLAUNCHTOKEN/USDT:USDT', 'ALU/USDT:USDT', 'ANIME/USDT:USDT', 'ANKR/USDT:USDT', 'APE/USDT:USDT', 'API3/USDT:USDT', 'APT/USDT:USD